In [1]:
import org.apache.spark.sql.types._
import spark.implicits._ 
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Vectors, Vector}

Intitializing Scala interpreter ...

Spark Web UI available at http://toshiba:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1614100294678)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import spark.implicits._
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.{Vectors, Vector}


In [9]:
val schema = StructType(
  StructField("Platform", StringType, nullable = true) ::
    StructField("Genre", StringType, nullable = true) ::
    StructField("Publisher", StringType, nullable = true) ::
    StructField("NA_Sales", DoubleType, nullable = true) ::
    StructField("EU_Sales", DoubleType, nullable = true) ::
    StructField("JP_Sales", DoubleType, nullable = true) ::
    StructField("Other_Sales", DoubleType, nullable = true) ::
    StructField("Global_Sales", DoubleType, nullable = true) ::
    StructField("Rating", StringType, nullable = true) ::    
    StructField("Critic_Score_Class", StringType, nullable = true) ::
    StructField("KMEANS-prediction", IntegerType, nullable = true) ::
    StructField("GMM-prediction", IntegerType, nullable = true) ::
    StructField("BisKMeans-prediction", IntegerType, nullable = true) ::
    Nil
)

schema: org.apache.spark.sql.types.StructType = StructType(StructField(Platform,StringType,true), StructField(Genre,StringType,true), StructField(Publisher,StringType,true), StructField(NA_Sales,DoubleType,true), StructField(EU_Sales,DoubleType,true), StructField(JP_Sales,DoubleType,true), StructField(Other_Sales,DoubleType,true), StructField(Global_Sales,DoubleType,true), StructField(Rating,StringType,true), StructField(Critic_Score_Class,StringType,true), StructField(KMEANS-prediction,IntegerType,true), StructField(GMM-prediction,IntegerType,true), StructField(BisKMeans-prediction,IntegerType,true))


In [10]:
val videogameDf = spark.read.format("csv").option("header", value = true).option("delimiter", ",").option("mode", "DROPMALFORMED").schema(schema).load("finalDF.csv").cache()

videogameDf.printSchema()
videogameDf.show(10)

root
 |-- Platform: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: double (nullable = true)
 |-- EU_Sales: double (nullable = true)
 |-- JP_Sales: double (nullable = true)
 |-- Other_Sales: double (nullable = true)
 |-- Global_Sales: double (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Critic_Score_Class: string (nullable = true)
 |-- KMEANS-prediction: integer (nullable = true)
 |-- GMM-prediction: integer (nullable = true)
 |-- BisKMeans-prediction: integer (nullable = true)

+--------+------+--------------------+--------+--------+--------+-----------+------------+------+------------------+-----------------+--------------+--------------------+
|Platform| Genre|           Publisher|NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Rating|Critic_Score_Class|KMEANS-prediction|GMM-prediction|BisKMeans-prediction|
+--------+------+--------------------+--------+--------+--------+-----------+------------

videogameDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Platform: string, Genre: string ... 11 more fields]


In [11]:
import org.apache.spark._
import org.apache.spark.rdd.RDD 
import org.apache.spark.util.IntParam 
import org.apache.spark.graphx._
import org.apache.spark.graphx.util.GraphGenerators 

import org.apache.spark._
import org.apache.spark.rdd.RDD
import org.apache.spark.util.IntParam
import org.apache.spark.graphx._
import org.apache.spark.graphx.util.GraphGenerators


In [77]:
var vertexArray:Array[(Long, String)] = new Array[(Long, String)](3)
vertexArray(0) = (0L, "NA_Sales")
vertexArray(1) = (1L, "EU_Sales")
vertexArray(2) = (2L, "JP_Sales")


vertexArray: Array[(Long, String)] = Array((0,NA_Sales), (1,EU_Sales), (2,JP_Sales))


vertexRDD: org.apache.spark.rdd.RDD[(Long, String)] = ParallelCollectionRDD[58] at parallelize at <console>:45


In [200]:
val length = videogameDf.count

length: Long = 4951


In [212]:
val edgeArray = videogameDf.select("NA_Sales", "EU_Sales", "JP_Sales").as[(Double,Double,Double)].collect
.flatMap{row => 
     val edges = Array(
                  Edge(0L, 1L, (row._1-row._2)),
                  Edge(0L, 2L, (row._1-row._3)),
                  Edge(1L, 2L, (row._2-row._3))
     )
    edges
}

edgeArray: Array[org.apache.spark.graphx.Edge[Double]] = Array(Edge(0,1,-0.05), Edge(0,2,-0.05), Edge(1,2,0.0), Edge(0,1,0.12000000000000001), Edge(0,2,0.13), Edge(1,2,0.01), Edge(0,1,0.13), Edge(0,2,0.33), Edge(1,2,0.2), Edge(0,1,0.18), Edge(0,2,0.38), Edge(1,2,0.2), Edge(0,1,0.16), Edge(0,2,0.13), Edge(1,2,-0.03), Edge(0,1,0.0), Edge(0,2,0.03), Edge(1,2,0.03), Edge(0,1,0.07999999999999999), Edge(0,2,0.0), Edge(1,2,-0.07999999999999999), Edge(0,1,-0.14), Edge(0,2,0.06), Edge(1,2,0.2), Edge(0,1,0.0), Edge(0,2,-0.01), Edge(1,2,-0.01), Edge(0,1,0.039999999999999994), Edge(0,2,0.04999999999999999), Edge(1,2,0.009999999999999995), Edge(0,1,-0.07), Edge(0,2,0.0), Edge(1,2,0.07), Edge(0,1,0.18), Edge(0,2,0.61), Edge(1,2,0.43), Edge(0,1,0.23999999999999996), Edge(0,2,0.10999999999999999), Edge...

In [217]:
val vertexRDD: RDD[(Long, (String))] = sc.parallelize(vertexArray)
val edgeRDD: RDD[Edge[Double]] = sc.parallelize(edgeArray)
val graph: Graph[(String), Double] = Graph(vertexRDD, edgeRDD)

edgeRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Double]] = ParallelCollectionRDD[192] at parallelize at <console>:45


graph: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@29012b51


In [244]:
graph.triplets.collect.filter{triplet => 
    (triplet.attr > 0 && triplet.srcAttr=="NA_Sales") ||
    (triplet.attr < 0 && triplet.dstAttr=="NA_Sales")}
.length

res145: Int = 7905


In [242]:
graph.triplets.collect.filter{triplet => 
    (triplet.attr > 0 && triplet.srcAttr=="JP_Sales") ||
    (triplet.attr < 0 && triplet.dstAttr=="JP_Sales")}
.length

res143: Int = 1038


In [243]:
graph.triplets.collect.filter{triplet => 
    (triplet.attr > 0 && triplet.srcAttr=="EU_Sales") ||
    (triplet.attr < 0 && triplet.dstAttr=="EU_Sales")}
.length

res144: Int = 4772
